In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import skimage.measure as skm
import skimage.transform as skt

import matplotlib.pyplot as plt
from random import seed
from random import random
import skimage.measure as skm
import os
from PIL import Image
import random
import pickle as pkl
import numpy as np
import argparse
import copy as cp
from sklearn.preprocessing import MultiLabelBinarizer





# Any results you write to the current directory are saved as output.

In [3]:
URL_ENDPOINT = "http://cs.mcgill.ca/~ksinha4/datasets/kaggle/"

train_x = np.loadtxt(URL_ENDPOINT+"train_x.csv", delimiter=",")
train_y = np.loadtxt(URL_ENDPOINT+"train_y.csv", delimiter=",")
test_x = np.loadtxt(URL_ENDPOINT+"test_x.csv", delimiter=",")


In [7]:
#test_y = np.loadtxt("submission.csv", delimiter = ",")

ValueError: could not convert string to float: 'Id'

In [8]:

def find_max_center(images):
    images= images.reshape(64, 64) 
    image = cp.deepcopy(images)
    for i in range(len(image)):
        for j in range(len(image[i])):
            if image[i][j] < 200:
                image[i][j] = 0
    inter = skm.regionprops(skm.label(image))
    ind_l = [j.area for j in inter]
    ind = ind_l.index(max(ind_l))
    return skt.resize(inter[ind].image,(64,64))


In [11]:

processed_x = [find_max_center(i) for i in train_x]


/home/peterpark/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [16]:
import math
import numpy as np
def sigmoid( x):
    return 1 / (1 + math.exp(-x * 1.0))

def dSigmoid(x):
    return x * (1.0 - x)

class edge:
    def __init__(self, connectedNeuron):
        self.connectedNeuron = connectedNeuron
        self.weight = np.random.normal()
        self.dWeight = 0.0
class Neuron:
    eta = 0.001
    alpha = 0.01
    def __init__(self, layer):
        self.dendrons = []
        self.error = 0.0
        self.gradient = 0.0
        self.output = 0.0
        if layer is None:
            pass
        else:
            for neuron in layer:
                con = edge(neuron)
                self.dendrons.append(con)
    def addError(self, err):
        self.error += err
    def setError(self, err):
        self.error = err
    def setOutput(self, output):
        self.output = output
    def feedForword(self):
        sumOutput = 0
        if len(self.dendrons) == 0:
            return
        for dendron in self.dendrons:
            sumOutput += dendron.connectedNeuron.output * dendron.weight
        self.output = sigmoid(sumOutput)
        
    def backPropagate(self):
        self.gradient = self.error * dSigmoid(self.output);
        for dendron in self.dendrons:
            dendron.dWeight = -Neuron.eta * (
            dendron.connectedNeuron.output * self.gradient) + self.alpha * dendron.dWeight;
            dendron.weight = dendron.weight + dendron.dWeight;
            dendron.connectedNeuron.addError(dendron.weight * self.gradient);
        self.error = 0;
class Network:
    def __init__(self, topology):
        self.layers = []
        for numNeuron in topology:
            layer = []
            for i in range(numNeuron):
                if (len(self.layers) == 0):
                    layer.append(Neuron(None))
                else:
                    layer.append(Neuron(self.layers[-1]))
            layer.append(Neuron(None))
            layer[-1].setOutput(1)
            self.layers.append(layer)

    def setInput(self, inputs):
        for i in range(len(inputs)):
            self.layers[0][i].setOutput(inputs[i])

    def feedForword(self):
        for layer in self.layers[1:]:
            for neuron in layer:
                neuron.feedForword();

    def backPropagate(self, target):
        for i in range(len(target)):
            self.layers[-1][i].setError(self.layers[-1][i].output - target[i] )
        for layer in self.layers[::-1]:
            for neuron in layer:
                neuron.backPropagate()
    def getError(self, target):
        err = 0
        for i in range(len(target)):
            e = (self.layers[-1][i].output -target[i] )
            err += e ** 2
        err = err / len(target)
        err = math.sqrt(err)
        return err
    def getResults(self):
        output = []
        for neuron in self.layers[-1]:
            output.append(neuron.output)
        output.pop()
        return output

    def getThResults(self):
        output = []
        for neuron in self.layers[-1]:
            o = neuron.output
            if (o > 0.5):
                o = 1
            else:
                o = 0
            output.append(o)
        output.pop()
        return output
    
topology = []
topology.append(4096)
topology.append(40)
topology.append(1)
net = Network(topology)
Neuron.eta = 0.09
Neuron.alpha = 0.015
epoch = 1
while epoch > 0:
    err = 0
    inputs = processed_x
    outputs = train_y
    for i in range(len(inputs)):
        net.setInput([item for sublist in np.ndarray.tolist(inputs[i]) for item in sublist])
        net.feedForword()
        net.backPropagate([outputs[i]])
        err += net.getError([outputs[i]])
    print("error: ", err)
    epoch -= 1
    if (err < 0.01):
        break

#while True:
#    a = input("type 1st input :")
#    b = input("type 2nd input :")
#    net.setInput([a, b])
#    net.feedForword()
#    print(net.getThResults())


error:  182849.46600246


In [ ]:
train_y